In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler


def normalize(data):
    
    normalizer = StandardScaler().fit(data)
    
    return normalizer.transform(data), normalizer

x = np.array([
    [100, 0.1, -4],
    [200, 0.3, -9],
    [500, 0.2, 0]
])
x_normed, x_normalizer = normalize(x)

x_normed

array([[-9.80580676e-01, -1.22474487e+00,  9.05357460e-02],
       [-3.92232270e-01,  1.22474487e+00, -1.26750044e+00],
       [ 1.37281295e+00, -3.39934989e-16,  1.17696470e+00]])

In [4]:
lenni = 3987
tr_re_va_te = np.array([0.4, 0.4, 0.1, 0.1])

N_train, N_recal, N_val, N_test = (lenni * tr_re_va_te).astype(int)
N_train                      

1594